In [1]:
pip install Dash

In [3]:
from dash import Dash, html, dcc
from dash.dependencies import Input, Output

# Crear la aplicación
app = Dash(__name__)

# Layout de la aplicación
app.layout = html.Div([
    html.H1("SpaceX Launch Dashboard", style={'textAlign': 'center'}),
    html.Div([
        html.Label("Select Launch Site:"),
        dcc.Dropdown(
            id='site-dropdown',
            options=[
                {'label': 'All Sites', 'value': 'ALL'},
                {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
            ],
            value='ALL',
            placeholder="Select a Launch Site here",
            searchable=True
        )
    ]),
    html.Div(id='output-container')  # Contenedor para mostrar resultados
])

# Callback para actualizar la salida
@app.callback(
    Output('output-container', 'children'),
    Input('site-dropdown', 'value')
)
def update_output(selected_site):
    if selected_site == 'ALL':
        return "Showing data for all sites."
    else:
        return f"Showing data for {selected_site}."

# Ejecutar la aplicación
if __name__ == '__main__':
    app.run_server(debug=True)


In [5]:
import pandas as pd

# Cargar el DataFrame (asegúrate de usar la ruta correcta a tu archivo)
spacex_df = pd.read_csv('spacex_launch_dash.csv')  # Ajusta la ruta a tu archivo

# Ahora el DataFrame `spacex_df` estará disponible en todo el código


In [9]:
from dash import Dash, html, dcc
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd

# Assuming spacex_df is already loaded in the app, here's an example:
# spacex_df = pd.read_csv('spacex_launch_data.csv')

# Create the Dash application
app = Dash(__name__)

# Layout of the app
app.layout = html.Div([
    html.H1("SpaceX Launch Success Dashboard", style={'textAlign': 'center'}),
    
    # Dropdown for selecting launch site
    html.Div([
        html.Label("Select Launch Site:"),
        dcc.Dropdown(
            id='site-dropdown',
            options=[
                {'label': 'All Sites', 'value': 'ALL'},
                {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
            ],
            value='ALL',  # Default value is 'ALL'
            placeholder="Select a Launch Site here",
            searchable=True
        )
    ]),
    
    # Container for the pie chart
    html.Div([
        dcc.Graph(id='success-pie-chart')
    ])
])

# Callback function to update the pie chart based on selected launch site
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value')
)
def get_pie_chart(entered_site):
    # Filter data based on selected site
    filtered_df = spacex_df
    if entered_site != 'ALL':
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]

    # Count the success (class = 1) and failure (class = 0) launches
    success_count = filtered_df[filtered_df['class'] == 1].shape[0]
    failure_count = filtered_df[filtered_df['class'] == 0].shape[0]

    # Prepare data for the pie chart
    labels = ['Success', 'Failure']
    values = [success_count, failure_count]
    
    # Create pie chart using Plotly
    fig = px.pie(
        names=labels,
        values=values,
        title=f'Success vs Failure Launches ({entered_site})'
    )

    return fig

# Run the application
if __name__ == '__main__':
    app.run_server(debug=True, port=8051)



In [15]:
import dash
from dash import dcc, html
import pandas as pd

# Suponiendo que ya tienes cargado el DataFrame spacex_df
# Por ejemplo, calculamos el rango de payload
min_payload = spacex_df['Payload Mass (kg)'].min()
max_payload = spacex_df['Payload Mass (kg)'].max()

# Inicializar la app de Dash
app = dash.Dash()

# Diseño de la aplicación
app.layout = html.Div([
    # Título de la sección
    html.H3("Selecciona el rango de carga útil para la misión"),

    # Rango deslizante (Slider)
    dcc.RangeSlider(
        id='payload-slider',
        min=0, 
        max=10000, 
        step=1000,
        marks={i: f'{i}' for i in range(0, 10001, 1000)},  # Marcas en cada intervalo de 1000 kg
        value=[min_payload, max_payload],  # Rango inicial
    ),

    # Muestra el valor actual seleccionado por el slider
    html.Div(id='output-range', style={'padding': '20px'})
])

# Callback para actualizar el valor seleccionado del slider
@app.callback(
    dash.dependencies.Output('output-range', 'children'),
    [dash.dependencies.Input('payload-slider', 'value')]
)
def update_output(value):
    return f"Rango seleccionado: {value[0]} kg - {value[1]} kg"

# Ejecutar la aplicación
if __name__ == '__main__':
    app.run_server(debug=True)


In [23]:
import dash
from dash import dcc, html
import plotly.express as px
import pandas as pd

# Suponiendo que ya tienes cargado el DataFrame spacex_df
# Suponiendo que spacex_df tiene las columnas 'Payload Mass (kg)', 'class', 'Booster Version Category', 'Launch Site'

# Inicializar la app de Dash
app = dash.Dash()

# Crear el layout
app.layout = html.Div([
    html.H3("Selecciona el rango de carga útil y el sitio de lanzamiento"),
    
    # Dropdown para seleccionar el sitio de lanzamiento
    dcc.Dropdown(
        id='site-dropdown',
        options=[
            {'label': 'Todos los sitios', 'value': 'ALL'},
            {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
            {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
            {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
            {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
        ],
        value='ALL',  # Valor inicial seleccionado
        multi=False
    ),
    
    # Rango deslizante para seleccionar el rango de carga útil
    dcc.RangeSlider(
        id='payload-slider',
        min=0, 
        max=10000, 
        step=1000,
        marks={i: f'{i}' for i in range(0, 10001, 1000)},  # Marcas en cada intervalo de 1000 kg
        value=[spacex_df['Payload Mass (kg)'].min(), spacex_df['Payload Mass (kg)'].max()]  # Valor inicial
    ),

    # Gráfico de dispersión para mostrar Payload y clase
    dcc.Graph(id='success-payload-scatter-chart')
])

# Callback para actualizar el gráfico de dispersión
@app.callback(
    dash.dependencies.Output('success-payload-scatter-chart', 'figure'),
    [
        dash.dependencies.Input('site-dropdown', 'value'),
        dash.dependencies.Input('payload-slider', 'value')
    ]
)
def update_scatter_plot(selected_site, payload_range):
    # Filtrar los datos según el rango de carga útil
    filtered_df = spacex_df[
        (spacex_df['Payload Mass (kg)'] >= payload_range[0]) & 
        (spacex_df['Payload Mass (kg)'] <= payload_range[1])
    ]

    # Si no se selecciona un sitio específico, usamos todos los sitios
    if selected_site != 'ALL':
        filtered_df = filtered_df[filtered_df['Launch Site'] == selected_site]

    # Crear el gráfico de dispersión
    fig = px.scatter(
        filtered_df,
        x='Payload Mass (kg)',
        y='class',
        color='Booster Version Category',  # Colorear por la versión del booster
        labels={'Payload Mass (kg)': 'Carga Útil (kg)', 'class': 'Resultado de Misión'},
        title=f"Dispersión de Carga Útil vs Resultado de Misión para {selected_site if selected_site != 'ALL' else 'Todos los sitios'}"
    )

    # Devolver la figura
    return fig

# Ejecutar la aplicación
if __name__ == '__main__':
    app.run_server(debug=True)


In [26]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd

# Cargar datos
spacex_df

# Inicializar la aplicación Dash
app = dash.Dash(__name__)

# Layout de la aplicación
dropdown_options = [{'label': 'All Sites', 'value': 'ALL'}] + [
    {'label': site, 'value': site} for site in spacex_df['Launch Site'].unique()
]

app.layout = html.Div([
    dcc.Dropdown(id='site-dropdown', options=dropdown_options, value='ALL', placeholder="Select a Launch Site"),
    dcc.RangeSlider(id='payload-slider', min=0, max=10000, step=1000, value=[0, 10000],
                    marks={i: str(i) for i in range(0, 11000, 2000)}),
    dcc.Graph(id='success-payload-scatter-chart')
])

# Callback para actualizar el gráfico
@app.callback(
    Output('success-payload-scatter-chart', 'figure'),
    [Input('site-dropdown', 'value'),
     Input('payload-slider', 'value')]
)
def update_scatter_chart(selected_site, payload_range):
    filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= payload_range[0]) &
                            (spacex_df['Payload Mass (kg)'] <= payload_range[1])]
    
    if selected_site == 'ALL':
        fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', color='Booster Version Category',
                         title='Correlation between Payload and Success for All Sites')
    else:
        site_df = filtered_df[filtered_df['Launch Site'] == selected_site]
        fig = px.scatter(site_df, x='Payload Mass (kg)', y='class', color='Booster Version Category',
                         title=f'Correlation between Payload and Success for {selected_site}')
    
    return fig

# Ejecutar la aplicación
if __name__ == '__main__':
    app.run_server(debug=True)
